In [22]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.metrics import precision_score, roc_auc_score, recall_score
from sklearn.tree import DecisionTreeClassifier

### Reading in cleaned datasets

In [23]:
words_df = pd.read_csv('../Data/words_df.csv')
words_df1 = pd.read_csv('../Data/words_df1.csv')

## PCA

In [24]:
#countvetorized
X = words_df.drop(columns = 'target')
y = words_df['target']

In [25]:
#tfidif
X1 = words_df1.drop(columns = 'target')
y1 = words_df1['target']

In [9]:
words_df.head()

,aba,aba woman,abandoned,abc,abc news,ablaze,able,absolutely,accident,according,...,youtube,youtube playlist,youtube video,yr,yr old,yyc,zone,subjectivity,polarity,target
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0000,0.00000,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.4000,0.10000,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.3875,-0.01875,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0000,0.00000,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0000,0.00000,1


In [26]:
#no NLP
X2 = words_df.drop(columns = ['target', 'subjectivity', 'polarity'])
y2 = words_df['target']

In [11]:
y.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify = y)

In [28]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, random_state=42, stratify = y)

In [29]:
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

In [15]:
from sklearn.decomposition import PCA

In [16]:
pca = PCA()
pca.fit(X_train)
Z_train = pca.transform(X_train)
Z_test = pca.transform(X_test)

In [17]:
var_exp = pca.explained_variance_ratio_
cum_var_exp = np.cumsum(var_exp)
cum_var_exp

array([0.01063268, 0.02107219, 0.03127022, ..., 1.        , 1.        ,
       1.        ])

# Modeling

In [30]:
logreg =LogisticRegression(solver = 'liblinear')

In [31]:
params_logreg = {'penalty': ['l1', 'l2']}

### Logisitic Regression with Scaling

In [32]:
gs_logreg = GridSearchCV(logreg, params_logreg, cv = 5)
gs_logreg.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='liblinear',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=None, param_grid={'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [35]:
# Model Evaluation
# print out the best score
print(gs_logreg.best_score_)
#print out the set of hyperparameters that achieved the best score.
logreg_model = gs_logreg.best_estimator_
print('accuracy score - Existing data', logreg_model.score(X_train, y_train))
print('accuracy score - New data',  logreg_model.score(X_test, y_test)) # Accuracy score 
print(roc_auc_score(y_test, logreg_model.predict_proba(X_test)[:,1]))
print(precision_score(y_test, logreg_model.predict(X_test)))

0.7416360133123139
accuracy score - Existing data 0.8954282711508145
accuracy score - New data 0.7605042016806722
0.8228599602858339
0.7201946472019465


In [126]:
file_name = '../Assets/logreg.sav'
pickle.dump(logreg_model, open(file_name, 'wb'))

### Logisitic Regressio with Tfidf + Scaling

In [36]:
gs_logreg1 = GridSearchCV(logreg, params_logreg, cv = 5)
gs_logreg1.fit(X_train1, y_train1)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='liblinear',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=None, param_grid={'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [37]:
# Model Evaluation
# print out the best score
print(gs_logreg1.best_score_)
#print out the set of hyperparameters that achieved the best score.
logreg_model1 = gs_logreg1.best_estimator_
print('accuracy score - Existing data', logreg_model1.score(X_train1, y_train1))
print('accuracy score - New data',  logreg_model1.score(X_test1, y_test1)) # Accuracy score 
print(roc_auc_score(y_test, logreg_model1.predict_proba(X_test1)[:,1]))
print(precision_score(y_test, logreg_model1.predict(X_test1)))

0.7773690663864075
accuracy score - Existing data 0.8246628131021194
accuracy score - New data 0.8051470588235294
0.8669986311670652
0.8170212765957446


In [125]:
file_name = '../Assets/logreg_tf_ss.sav'
pickle.dump(logreg_model1, open(file_name, 'wb'))

### Logisitic Regression with Scaling + PCA

In [107]:
#gs_logreg1 = GridSearchCV(logreg, params_logreg, cv = 5)
#gs_logreg1.fit(Z_train, y_train)

In [108]:
# Model Evaluation
# print out the best score
#print(gs_logreg1.best_score_)
#print out the set of hyperparameters that achieved the best score.
#logreg_model1 = gs_logreg1.best_estimator_
#print('accuracy score - Existing data', logreg_model1.score(Z_train, y_train))
#print('accuracy score - New data',  logreg_model1.score(Z_test, y_test)) # Accuracy score 

___

### Randomforest with Scaling

In [53]:
rf = RandomForestClassifier()

In [54]:
rf_params = {
    'n_estimators': [100, 150, 200],
    'max_depth': [None, 5, 10], 
    'min_samples_split' : [2,3, 5]
}
gs_rf = GridSearchCV(rf, param_grid=rf_params, cv=5)
gs_rf.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

In [55]:
print(gs_rf.best_score_)
#print out the set of hyperparameters that achieved the best score.
rf_model = gs_rf.best_estimator_
print('accuracy score - Existing data', rf_model.score(X_train, y_train))
print('accuracy score - New data', rf_model.score(X_test, y_test)) # Accuracy 

0.7852521292810098
accuracy score - Existing data 0.9570853039061131
accuracy score - New data 0.7914915966386554


In [61]:
roc_auc_score(y_test, rf_model.predict_proba(X_test)[:,1])

0.8472895757068175

In [71]:
precision_score(y_test, rf_model.predict(X_test))

0.8100147275405007

In [56]:
file_name = '../Assets/rf_ss.sav'
pickle.dump(rf_model, open(file_name, 'wb'))

### Randomforest with Scaling + Tfidf

In [63]:
gs_rf1 = GridSearchCV(rf, param_grid=rf_params, cv=5)
gs_rf1.fit(X_train1, y_train1)

GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

In [64]:
print(gs_rf1.best_score_)
#print out the set of hyperparameters that achieved the best score.
rf_model1 = gs_rf1.best_estimator_
print('accuracy score - Existing data', rf_model1.score(X_train1, y_train1))
print('accuracy score - New data', rf_model1.score(X_test1, y_test1)) # Accuracy 

0.7735152591437443
accuracy score - Existing data 0.977228936766509
accuracy score - New data 0.792016806722689


In [67]:
roc_auc_score(y_test1, rf_model1.predict_proba(X_test1)[:,1])

0.8390157911088898

In [72]:
precision_score(y_test1, rf_model1.predict(X_test1))

0.8177710843373494

In [65]:
file_name = '../Assets/rf_ss_tf.sav'
pickle.dump(rf_model1, open(file_name, 'wb'))

### Ada Boost with tfidf

In [45]:
ada = AdaBoostClassifier(base_estimator=DecisionTreeClassifier())

In [46]:
ada_params = {
    'n_estimators': [50,100],
    'base_estimator__max_depth': [1,2],
    'learning_rate': [.9, 1.]
}

In [47]:
gs_ada = GridSearchCV(ada, param_grid=ada_params, cv=5)
gs_ada.fit(X_train1, y_train1)

GridSearchCV(cv=5, error_score=nan,
             estimator=AdaBoostClassifier(algorithm='SAMME.R',
                                          base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                                                class_weight=None,
                                                                                criterion='gini',
                                                                                max_depth=None,
                                                                                max_features=None,
                                                                                max_leaf_nodes=None,
                                                                                min_impurity_decrease=0.0,
                                                                                min_impurity_split=None,
                                                                                min_samples_leaf=1,
 

In [48]:
print(gs_ada.best_score_)
#print out the set of hyperparameters that achieved the best score.
ada_model = gs_ada.best_estimator_
print('accuracy score - Existing data', ada_model.score(X_train1, y_train1))
print('accuracy score - New data', ada_model.score(X_test1, y_test1)) # Accuracy 

0.7565261369339888
accuracy score - Existing data 0.8481345244351025
accuracy score - New data 0.7778361344537815


In [51]:
roc_auc_score(y_test1, ada_model.predict_proba(X_test1)[:,1])

0.8212648646701517

In [70]:
precision_score(y_test1, ada_model.predict(X_test1))

0.7731673582295989

In [49]:
file_name = '../Assets/ada_tf.sav'
pickle.dump(ada_model, open(file_name, 'wb'))

## Multinomial Naive Beyes

In [20]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, random_state=42, stratify = y)

In [20]:
file_name = '../Assets/words_df_columns.pkl'
pickle.dump(X_train2.columns, open(file_name, 'wb'))

In [16]:
mnb = MultinomialNB()

In [17]:
mnb_model = mnb.fit(X_train2, y_train2)

In [18]:
print(mnb_model.score(X_train2, y_train2))
print(mnb_model.score(X_test2, y_test2))

0.8131021194605009
0.8014705882352942


In [38]:
roc_auc_score(y_test2, mnb_model.predict_proba(X_test2)[:,1])

0.8546808232809664

In [69]:
precision_score(y_test2, mnb_model.predict(X_test2))

0.8225806451612904

In [19]:
file_name = '../Assets/multi_bayes.sav'
pickle.dump(mnb_model, open(file_name, 'wb'))

## Gaussian NB

In [22]:
gnb = GaussianNB()
gnb_model = gnb.fit(X_train2, y_train2)

In [23]:
print(gnb_model.score(X_train2, y_train2))
print(gnb_model.score(X_test2, y_test2))

0.74373795761079
0.7342436974789915


In [36]:
roc_auc_score(y_test2, gnb_model.predict_proba(X_test2)[:,1])

0.7276619072705742

In [68]:
precision_score(y_test2, gnb_model.predict(X_test2))

0.9020618556701031

In [24]:
file_name = '../Assets/gaussian.sav'
pickle.dump(gnb_model, open(file_name, 'wb'))

## Metrics

In [2]:
import pickle

In [38]:
gnb_model = pickle.load(open('../Assets/gaussian.sav', 'rb'))
mnb_model = pickle.load(open('../Assets/multi_bayes.sav', 'rb'))
ada_model = pickle.load(open('../Assets/ada_tf.sav', 'rb'))
logreg_model = pickle.load(open('../Assets/logreg.sav', 'rb'))
logreg_model1 = pickle.load(open('../Assets/logreg_tf_ss.sav', 'rb'))
rf_model = pickle.load(open('../Assets/rf_ss.sav', 'rb'))
rf_model1 = pickle.load(open('../Assets/rf_ss_tf.sav', 'rb'))

/Users/brendan/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator GaussianNB from version 0.22.1 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/brendan/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator MultinomialNB from version 0.22.1 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


ModuleNotFoundError: No module named 'sklearn.ensemble._weight_boosting'